In [ ]:
import glob
import pickle

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

# %matplotlib inline

# Transform

In [ ]:
images_directory = 'test_images/'
image_names_prefix = 'straight_lines'
image_file_extension = '.jpg'
images = glob.glob(images_directory + image_names_prefix + '*' + image_file_extension)

pickle_path = 'wide_dist_pickle.p'
with open(pickle_path, mode='rb') as f:
    dist_pickle = pickle.load(f)
    
mtx, dist = dist_pickle['mtx'], dist_pickle['dist']

src_pt1 = [595, 450]
src_pt2 = [687, 450]
src_pt3 = [1100, 720]
src_pt4 = [200, 720]
src = np.float32([src_pt1, src_pt2, src_pt3, src_pt4])

dst_pt1 = [200, 100]
dst_pt2 = [1100, 100]
dst_pt3 = [1100, 720]
dst_pt4 = [200, 720]
dst = np.float32([dst_pt1, dst_pt2, dst_pt3, dst_pt4])

In [ ]:
def draw_lines(img, points, color=[255, 0, 0], thickness=2):
    pt1 = points[0]
    pt2 = points[1]
    pt3 = points[2]
    pt4 = points[3]

    cv2.line(img, (pt1[0], pt1[1]), (pt2[0], pt2[1]), color, thickness)
    cv2.line(img, (pt2[0], pt2[1]), (pt3[0], pt3[1]), color, thickness)
    cv2.line(img, (pt3[0], pt3[1]), (pt4[0], pt4[1]), color, thickness)
    cv2.line(img, (pt4[0], pt4[1]), (pt1[0], pt1[1]), color, thickness)

In [ ]:
# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 6))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images')
    
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    draw_lines(img, src)
    sub2.imshow(img)
    if idx == 0:
        sub2.set_title('Original Images with source points')

fig.tight_layout()
plt.show()

In [ ]:
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 6))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    img_size = img.shape[1::-1]
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    draw_lines(img, src)
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images with source points')

    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    draw_lines(warped, dst)
    sub2.imshow(warped)
    if idx == 0:
        sub2.set_title('Transformed Images with destination points')

fig.tight_layout()
plt.show()

# Testing

In [ ]:
images = glob.glob('test_images/test*.jpg')

fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)

    img_size = img.shape[1::-1]
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.set_xticks(())
    sub1.set_yticks(())
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images')

    # Draw and display the undistorted images
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.set_xticks(())
    sub2.set_yticks(())
    sub2.imshow(warped)
    if idx == 0:
        sub2.set_title('Transformed Images')

fig.tight_layout()
plt.show()